# LLM Fine-Tuning (gemma-2b + LoRA)

공개된 대규모 언어모델 **"gemma-2b"** 을 Foundation Model로 하고, LoRA 기법을 적용하여 효과적으로 파인튜닝하는 실습을 진행해 보겠습니다.  
- Task: Causal Language Model (Instruct Fine-Tuning)
- Foundation Model: “gemma-2b”
- Dataset: “KorAlpaca”
- Trainer: Huggingface PEFT/LoRA

## 0. Setup

In [1]:
import os
os.environ['HF_HOME'] = 'D:/HF/cache'
os.environ['HF_DATASETS'] = 'D:/HF/datasets'
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = "1"

In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["FSDP_CPU_RAM_EFFICIENT_LOADING"] = "false"

In [ ]:
# !pip install -q transformers==4.51.1
# !pip install -q datasets==3.2.0
# !pip install -q peft==0.15.1
# !pip install -q trl==0.16.1
# !pip install -q accelerate==1.3.0

In [ ]:
# !pip install -q bitsandbytes

In [3]:
from IPython.display import display
from tqdm.notebook import tqdm as notebook_tqdm

In [4]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, pipeline
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
"""
from huggingface_hub import notebook_login
notebook_login()
"""

In [ ]:
"""
from huggingface_hub import login
from dotenv import load_dotenv
from pathlib import Path
import os

dotenv_path = Path('Z:/Misc/access_token.env')
load_dotenv(dotenv_path=dotenv_path)
access_key=os.getenv('HF_TOKEN')

print(f"HF Access Key: {access_key}")
"""

In [5]:
!nvidia-smi

Fri Aug  1 13:12:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 553.62                 Driver Version: 553.62         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q               WDDM  |   00000002:00:00.0 Off |                    0 |
| N/A    0C    P8             N/A /  N/A  |    1426MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 1. DataSet: KoAlpaca v1.1a

질문을 전달하면 답변하는 형태로 대규모 언어모델을 파인튜닝하기 위한 학습 데이터셋을 준비하겠습니다.  
데이터셋은 **Instruction** (지시사항)과 **Output** (출력)의 쌍으로 구성되어 있습니다.  
KoAlpaca 데이터셋은 지식iN 기반의 질문-답변 데이터셋이며, Huggingface Dataset 에 공개되어 있습니다.

In [6]:
from datasets import load_dataset

# dataset = load_dataset("beomi/KoAlpaca-v1.1a")
dataset = load_dataset("//swschoolavdazfiles002.file.core.windows.net/aias-language/Dataset/KoAlpaca-v1.1a")

dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url'],
        num_rows: 21155
    })
})

In [7]:
dataset['train'][0]

{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.',
 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268'}

파인튜닝하기 위하여 학습 데이터를 다음과 같은 Instruction Format 형태로 변환을 합니다.  
```
### Question:

### Response: <eos>
```

In [8]:
def generate_prompt(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        # [실습] 다음 코드를 완성하세요!!
        # Example에 대해 위에서 정의한 Instruction Format 형태로 변환합니다.
        prompt = f"### Question: {example['instruction'][i]}\n\n### Response: {example['output'][i]}<eos>"
        output_texts.append(prompt)
    return output_texts

In [9]:
train_data = dataset['train']
print(generate_prompt(train_data[:1])[0])

### Question: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?

### Response: 양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. 

식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.

 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? 

고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.<eos>


## 2. Foundation Model & Tokenizer

Foundation LLM 으로 "google/gemma-2b" 모델을 로딩합니다.  
- Lightweight, state-of-the-art open models
- Text-to-text, decoder-only LLM (English)
- Instruction following & Multi-turn conversations

In [10]:
# BASE_MODEL = "google/gemma-2b"
BASE_MODEL = "//swschoolavdazfiles002.file.core.windows.net/aias-language/Model/gemma-2b"

# [실습] 다음 코드를 완성하세요!!
# 사전 학습된 'google/gemma-2b' 모델과 토크나이저를 가져옵니다.
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.padding_side = 'right'

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
prompt = "건강하게 살기 위한 세 가지 방법은?"

In [12]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

# [실습] 다음 코드를 완성하세요!!
# Text-Generation Pipeline 관련 do_sample, temperature, top_k, top_p, repetition_penalty 등 파라미터를 설정합니다.
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
)

print(outputs[0]["generated_text"][len(prompt):])

Device set to use cuda:0




1. <strong>운동</strong>
2. <strong>식사</strong>
3. <strong>정신적 안녕감</strong>

이 중에서 가장 중요한 것은 운동이다. 운동을 하지 않으면 건강하지 못하고, 운동을 하면 건강해진다. 그렇지만 운동을 하는데 있어서 무엇이 중요할까요?

<h2>무엇이 운동의 주된 목표인가</h2>

운동의 목표는 어떤 것이 있는가?

* 체중 조절
* 근력 강화
* 심장과 호흡기 기능 향상
* 신체적 활동 증진
* 정신적 안녕감 증진
* 생리적 안녕감 증진
* 성장 발달에 도움
* 삶의 질 향상

그런데 이 중에서 운동을 하는데 있어서 가장 중요한 것은 무엇일까요?

<h3>심장과 호흡기 기능 향상</h3>

우선적으로 생각해볼 수 있는 것은 심장과 호흡기 기능 향상이다.

심장과 호흡기 기능 향상을 위해서는 운동을 해야 한다.

운동을 하지 않는다면 심장과 호흡기 기능이 저하되고, 심장과 호흡기 기능이 저하되면 건강이 나빠질 것이다.

운동을 하지 않으면 심장과 호흡기 기능이 저하되고, 심장과 호흡기 기능이 저하되면 건강이 나빠질 것이다.

그래서 운동을 해야 한다.

<h3>근육 강화</h3>

또 다른 운동의 목표는 근육 강화이다.

근육 강화를 위해서는 운동을 해야 한다.

근육 강화를 위해서는 운동을 해야 한다.

근육 강화를 위해서는 운동을 해야 한다.

근육 강화를 위해서는 운동을 해야 한다.

근육 강화를 위해서는 운동을 해야 한다.

근육 강화를 위해서는 운동을 해야 한다.

근육 강화를 위해서는 운동을 해야 한다.

근육 강화를 위해서는 운동을 해야 한다.

근육 강화를 위해서는 운동을 해야 한다.

근육 강화


## 3. PEFT LoRA Setup

LoraConfig 함수를 통해 파인튜닝을 위한 주요 LoRA 파라미터를 설정합니다.

In [13]:
# [실습] 다음 코드를 완성하세요!!
# LoRA 파인튜닝을 위한 Config를 설정합니다. r, lora_alpha, lora_dropout, target_modules, task_type
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    task_type="CAUSAL_LM",
)

## 4. Model Training (PEFT LoRA)

SFT Trainer 관련 필요한 학습 파라미터 설정한 후 PEFT LoRA 기반으로 파인튜닝을 위한 학습을 진행합니다.

In [15]:
# [실습] 다음 코드를 완성하세요!!
# Training Arguments를 설정을 합니다. (num_train_epochs, max_steps,warmup_steps, etc.)
training_args = TrainingArguments(
    output_dir="D:/Trainer/PEFT-Trainer",
    num_train_epochs = 1,
    max_steps=1000, # 3000
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    label_names=["labels"],
    # optim="adamw_torch", # Default: Pytorch AdamW
    warmup_steps=10,
    learning_rate=2e-4,
    bf16=True, # fp16=True
    logging_steps=100,
    push_to_hub=False,
    report_to='none',
    dataloader_num_workers=2,
    dataloader_prefetch_factor=1,
)

In [16]:
# [실습] 다음 코드를 완성하세요!!
# 파인튜닝을 위한 Trainer 설정을 합니다. (model, train_dataset, args, peft_config, formatting_func)
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    args=training_args,
    peft_config=lora_config,
    formatting_func=generate_prompt,
    processing_class=tokenizer,
)

In [17]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

LoRA 모델을 통해 학습 가능한 파라미터수는 전체 모델 파라미터(2,513,526,784)의 0.05% 수준입니다.

In [18]:
print_trainable_parameters(model)

trainable params: 1382400 || all params: 2507554816 || trainable%: 0.05512940300165307


1000 스텝 학습에 10분 정도 소요되니, TrainingArguments에서 **`max_steps`** 설정하실 때 참고하시기 바랍니다.

In [19]:
trainer.train()

Step,Training Loss
100,1.928400
200,1.852400
300,1.834500
400,1.825100
500,1.823900
600,1.837900
700,1.846200
800,1.842000
900,1.838700
1000,1.825500


TrainOutput(global_step=1000, training_loss=1.8454728851318358, metrics={'train_runtime': 552.2234, 'train_samples_per_second': 7.243, 'train_steps_per_second': 1.811, 'total_flos': 1.4116651237736448e+16, 'train_loss': 1.8454728851318358})

학습된 LoRA Adapter 모델을 Local Directory 에 저장합니다.

In [ ]:
"""
ADAPTER_MODEL = "lora_adapter"

trainer.model.save_pretrained(ADAPTER_MODEL)
"""

LoRA 학습된 weight을 원래 gemma-2b 모델과 합쳐 하나의 모델로 만들겠습니다.

In [ ]:
"""
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.bfloat16)
# model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.bfloat16)
model.load_adapter(ADAPTER_MODEL)

model = model.merge_and_unload()
model.save_pretrained('gemma-2b-peft')
"""

## 5. Model Inference

파인튜닝된 모델을 이용하여 Generation 해 보도록 하겠습니다.

In [20]:
pipe_finetuned = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

Device set to use cuda:0


In [21]:
prompt = "건강하게 살기 위한 세 가지 방법은?"
formatted_prompt = f"### Question: {prompt}\n\n### Response:"

In [22]:
outputs = pipe_finetuned(
    formatted_prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
)
print(outputs[0]["generated_text"][len(formatted_prompt):])

 1. 체중 관리
체중을 조절하는 것은 건강한 생활을 유지하기 위해서 가장 중요합니다. 적정 체중을 유지하면 혈압이 안 좋아져서 심장질환이나 당뇨병 등의 질병을 예방할 수 있습니다. 또한, 비만으로 인해 발생하는 여러 가지 문제를 해결할 수 있고, 운동과 식사조절을 통해 체중을 조절할 수 있습니다.

2. 스트레스 관리
스트레스는 건강에 부정적인 영향을 미치므로, 스트레스를 줄이는 것이 중요합니다. 스트레스를 줄이려면 우선적으로 자아존중감을 높여야 합니다. 이와 함께 정신적, 신체적 활동을 통해 스트레스를 완화시키며, 긍정적인 생각을 바르게 하여 스트레스를 감소시킬 수 있습니다.

3. 휴식 및 여가 활동
휴식 및 여가 활동은 건강한 생활을 유지하는데 있어 매우 중요합니다. 휴식은 피로를 없애고, 신체적, 정신적 회복을 도모하며, 몸과 마음을 즐겁게 하는 데 큰 역할을 합니다. 여가 활동은 육체적, 정신적 건강을 증진시켜주며, 사회적 관계를 형성하고, 새로운 경험을 맛보는 기회를 제공합니다.


- Ref. https://huggingface.co/docs/peft/main/en/task_guides/token-classification-lora